In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import difflib
import fuzzymatcher

In [2]:
import gc

In [3]:
j_df = pd.read_csv('jr.csv', sep=';')

In [53]:
j_df

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2017),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Publisher,Categories
0,1,12137,Journal of Statistical Software,journal,15487660,"13,802",Q1,103,84,262,3403,4244,262,"19,99","40,51",United States,University of California at Los Angeles,Software (Q1); Statistics and Probability (Q1)...
1,2,4700152228,Molecular Systems Biology,journal,17444292,"8,504",Q1,123,63,186,3816,1446,168,"7,51","60,57",United States,Wiley-Blackwell,Agricultural and Biological Sciences (miscella...
2,3,17945,Bioinformatics,journal,"13674811, 13674803","6,140",Q1,318,726,2360,17150,18335,2313,"5,48","23,62",United Kingdom,Oxford University Press,Biochemistry (Q1); Computational Mathematics (...
3,4,19169,Journal of Operations Management,journal,02726963,"5,739",Q1,158,19,139,1403,843,128,"5,16","73,84",Netherlands,Elsevier BV,Computer Science Applications (Q1); Industrial...
4,5,21100200203,Wiley Interdisciplinary Reviews: Computational...,journal,"17590884, 17590876","5,698",Q1,53,39,102,5480,1376,98,"8,97","140,51",United States,John Wiley & Sons Inc.,Biochemistry (Q1); Computational Mathematics (...
5,6,144974,Journal of Service Research,journal,10946705,"5,247",Q1,97,29,91,1922,709,87,"7,12","66,28",United States,SAGE Publications,Information Systems (Q1); Organizational Behav...
6,7,12402,MIS Quarterly: Management Information Systems,journal,02767783,"5,085",Q1,189,59,147,4974,1214,141,"7,05","84,31",United States,MIS Research Center,Computer Science Applications (Q1); Informatio...
7,8,21100420802,GigaScience,journal,2047217X,"5,022",Q1,24,123,150,5217,1012,122,"8,27","42,41",United Kingdom,Oxford University Press,Computer Science Applications (Q1); Health Inf...
8,9,100147317,Journal of Supply Chain Management,journal,15232409,"5,015",Q1,71,21,67,1676,470,63,"6,10","79,81",United States,Wiley-Blackwell,"Economics, Econometrics and Finance (miscellan..."
9,10,17700155018,Handbook of Computational Economics,book series,15740021,"4,160",Q1,20,0,13,0,20,11,"0,00","0,00",Netherlands,Elsevier BV,Computer Science Applications (Q1); Economics ...


In [5]:
j_df.columns[j_df.isnull().any()].tolist()

['SJR', 'Publisher']

In [6]:
sjr_null_count = j_df['SJR'].isnull().sum()
pub_null_count = j_df['Publisher'].isnull().sum()
sjr_null_count, pub_null_count

(600, 5634)

In [7]:
# Replace nulls in SJR with zeroes, this is what the Scimagojr website suggests.
# Eg: https://www.scimagojr.com/journalsearch.php?q=21100817106&tip=sid&clean=0
j_df.loc[j_df['SJR'].isnull(), 'SJR'] = 0

In [8]:
j_df

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2017),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Publisher,Categories
0,1,12137,Journal of Statistical Software,journal,15487660,"13,802",Q1,103,84,262,3403,4244,262,"19,99","40,51",United States,University of California at Los Angeles,Software (Q1); Statistics and Probability (Q1)...
1,2,4700152228,Molecular Systems Biology,journal,17444292,"8,504",Q1,123,63,186,3816,1446,168,"7,51","60,57",United States,Wiley-Blackwell,Agricultural and Biological Sciences (miscella...
2,3,17945,Bioinformatics,journal,"13674811, 13674803","6,140",Q1,318,726,2360,17150,18335,2313,"5,48","23,62",United Kingdom,Oxford University Press,Biochemistry (Q1); Computational Mathematics (...
3,4,19169,Journal of Operations Management,journal,02726963,"5,739",Q1,158,19,139,1403,843,128,"5,16","73,84",Netherlands,Elsevier BV,Computer Science Applications (Q1); Industrial...
4,5,21100200203,Wiley Interdisciplinary Reviews: Computational...,journal,"17590884, 17590876","5,698",Q1,53,39,102,5480,1376,98,"8,97","140,51",United States,John Wiley & Sons Inc.,Biochemistry (Q1); Computational Mathematics (...
5,6,144974,Journal of Service Research,journal,10946705,"5,247",Q1,97,29,91,1922,709,87,"7,12","66,28",United States,SAGE Publications,Information Systems (Q1); Organizational Behav...
6,7,12402,MIS Quarterly: Management Information Systems,journal,02767783,"5,085",Q1,189,59,147,4974,1214,141,"7,05","84,31",United States,MIS Research Center,Computer Science Applications (Q1); Informatio...
7,8,21100420802,GigaScience,journal,2047217X,"5,022",Q1,24,123,150,5217,1012,122,"8,27","42,41",United Kingdom,Oxford University Press,Computer Science Applications (Q1); Health Inf...
8,9,100147317,Journal of Supply Chain Management,journal,15232409,"5,015",Q1,71,21,67,1676,470,63,"6,10","79,81",United States,Wiley-Blackwell,"Economics, Econometrics and Finance (miscellan..."
9,10,17700155018,Handbook of Computational Economics,book series,15740021,"4,160",Q1,20,0,13,0,20,11,"0,00","0,00",Netherlands,Elsevier BV,Computer Science Applications (Q1); Economics ...


In [12]:
PICKLE_PATH = 'dblp_jr_df.pickle'
try:
    dblp_jr_df = pd.read_pickle(PICKLE_PATH)
except:
    dblp_jr_df1 = pd.read_pickle('./dblp_jr1.pickle')
    dblp_jr_df2 = pd.read_pickle('./dblp_jr2.pickle')
    dblp_jr_df3 = pd.read_pickle('./dblp_jr3.pickle')
    dblp_jr_df4 = pd.read_pickle('./dblp_jr4.pickle')
    dblp_jr_df5 = pd.read_pickle('./dblp_jr5.pickle')
    dblp_jr_df6 = pd.read_pickle('./dblp_jr6.pickle')
    dblp_jr_df7 = pd.read_pickle('./dblp_jr7.pickle')
    dblp_jr_df8 = pd.read_pickle('./dblp_jr8.pickle')
    dblp_jr_df9 = pd.read_pickle('./dblp_jr9.pickle')
    dblp_jr_df10 = pd.read_pickle('./dblp_jr10.pickle')
    dblp_jr_df11 = pd.read_pickle('./dblp_jr11.pickle')
    dblp_jr_df = pd.concat([dblp_jr_df1, dblp_jr_df2, dblp_jr_df3, dblp_jr_df4, dblp_jr_df5, dblp_jr_df6, dblp_jr_df7, 
                         dblp_jr_df8, dblp_jr_df9, dblp_jr_df10, dblp_jr_df11])
    dblp_jr_df.to_pickle(PICKLE_PATH)

In [11]:
# # Without a fuzzy join we only matched 622702 out of 3079007 rows. Now we consider a fuzzy join.

In [30]:
print(dblp_jr_df[(dblp_jr_df.id == '001c58d3-26ad-46b3-ab3a-c1e557d16821')]['venue'].values)
print(dblp_jr_df[(dblp_jr_df.id == '001c58d3-26ad-46b3-ab3a-c1e557d16821')]['Title'].values)

['visual analytics science and technology']
[ '2014 IEEE Conference on Visual Analytics Science and Technology, VAST 2014 - Proceedings']


In [32]:
print(dblp_jr_df[(dblp_jr_df.id == '074b2794-468e-4e3d-95ed-669a2e447217')]['venue'].values)
print(dblp_jr_df[(dblp_jr_df.id == '074b2794-468e-4e3d-95ed-669a2e447217')]['Title'].values)

['international conference on enterprise information systems']
[ 'ICEIS 2015 - 17th International Conference on Enterprise Information Systems, Proceedings']


In [36]:
print(dblp_jr_df[(dblp_jr_df.id == '04136c62-06a9-4c74-9da2-547448a9dc6f')]['venue'].values)
print(dblp_jr_df[(dblp_jr_df.id == '04136c62-06a9-4c74-9da2-547448a9dc6f')]['Title'].values)

['the european symposium on artificial neural networks']
[ '22nd European Symposium on Artificial Neural Networks, Computational Intelligence and Machine Learning, ESANN 2014 - Proceedings']


In [40]:
print(dblp_jr_df[(dblp_jr_df.id == '092edef1-8e7d-44eb-8126-49443b689aba')]['venue'].values)
print(dblp_jr_df[(dblp_jr_df.id == '092edef1-8e7d-44eb-8126-49443b689aba')]['Title'].values)

['the florida ai research society']
[ 'Proceedings of the 27th International Florida Artificial Intelligence Research Society Conference, FLAIRS 2014']


In [41]:
print(dblp_jr_df[(dblp_jr_df.id == '0ca08f64-95ad-451a-8c0b-31d82395b1df')]['venue'].values)
print(dblp_jr_df[(dblp_jr_df.id == '0ca08f64-95ad-451a-8c0b-31d82395b1df')]['Title'].values)

['international conference on wireless networks']
[ 'International Conference on Wireless Networks and Mobile Communications, WINCOM 2015']


In [42]:
print(dblp_jr_df[(dblp_jr_df.id == '0ef1cf4b-4010-4e3c-84e6-c389ba675bb8')]['venue'].values)
print(dblp_jr_df[(dblp_jr_df.id == '0ef1cf4b-4010-4e3c-84e6-c389ba675bb8')]['Title'].values)

['metadata and semantics research']
['International Journal of Metadata, Semantics and Ontologies']


In [44]:
print(dblp_jr_df[(dblp_jr_df.id == '0d9d0e38-d472-4dc9-84dd-76b38d8ac6b7')]['venue'].values)
print(dblp_jr_df[(dblp_jr_df.id == '0d9d0e38-d472-4dc9-84dd-76b38d8ac6b7')]['Title'].values)

['Theory and Practice of Object Systems']
['Theory of Computing Systems']


In [48]:
dblp_jr_df[(dblp_jr_df.best_match_score > 0.25)]

,best_match_score,__id_left,__id_right,abstract,authors,id,n_citation,references,title,venue,...,Total Docs. (2017),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Publisher,Categories
7,0.530366,1_left,1703_right,This paper describes the design and implementa...,"[Gareth Beale, Graeme Earl]",001c58d3-26ad-46b3-ab3a-c1e557d16821,50,"[10482dd3-4642-4193-842f-85f3b70fcf65, 3133714...",A methodology for the physically accurate visu...,visual analytics science and technology,...,0.0,109.0,0.0,105.0,107.0,"0,98","0,00",United States,NaN,Computer Science Applications; Computer Vision...
10,0.483278,2_left,45_right,This article applied GARCH model instead AR or...,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",001c8744-73c4-4b04-9364-22d31a10dbf1,50,"[2d84c0f2-e656-4ce7-b018-90eda1c132fe, a083a1b...","Comparison of GARCH, Neural Network and Suppor...",pattern recognition and machine intelligence,...,323.0,583.0,9478.0,7351.0,575.0,"11,38","29,34",United States,Institute of Electrical and Electronics Engineers,Applied Mathematics (Q1); Artificial Intellige...
22,0.696859,5_left,3380_right,NaN,"[Rafael Álvarez, Leandro Tortosa, José-Francis...",005ce28f-ed77-4e97-afdc-a296137186a1,0,NaN,COMPARING GNG3D AND QUADRIC ERROR METRICS METH...,international conference on computer graphics ...,...,0.0,57.0,0.0,23.0,55.0,"0,00","0,00",United States,NaN,Computer Graphics and Computer-Aided Design; C...
51,0.573528,9_left,4897_right,NaN,"[Dominik Szajerman, Adam Jurczyński]",00964544-cbe2-4da9-bb5a-03333160eb34,0,"[3fcd7cdc-20e6-4ea3-a41c-db126fcc5cfe, bf3a11c...",Fur Visualisation for Computer Game Engines an...,international conference on computer vision an...,...,0.0,58.0,0.0,21.0,55.0,"0,38","0,00",United States,NaN,Hardware and Architecture
210,0.891586,14_left,629_right,Three speech training systems for hearing-impa...,"[Minoru Shigenaga, Yoshihiro Sekiguchi]",00ca027b-5174-40fa-bf63-9a97c2a5f518,0,NaN,Speech training systems using lateral shapes o...,international joint conference on artificial i...,...,783.0,1304.0,19450.0,2949.0,1300.0,"2,27","24,84",United States,NaN,Artificial Intelligence
211,1.216361,15_left,1255_right,NaN,"[Reiko Heckel, Tom Mens, Michel Wermelinger]",00dc2bba-3237-4d4e-b541-1205b97df981,50,NaN,Software Evolution through Transformations.,Electronic Notes in Theoretical Computer Science,...,29.0,294.0,507.0,191.0,271.0,"0,60","17,48",Netherlands,Elsevier BV,Computer Science (miscellaneous) (Q2); Theoret...
214,0.970531,17_left,633_right,NaN,"[Milos Zelezný, Petr Císar, Zdenek Krnoul, Jan...",00e02aeb-b424-4ca8-b3ca-6e18e322f79e,7,NaN,Design of an audio-visual speech corpus for th...,conference of the international speech communi...,...,839.0,2220.0,19945.0,3979.0,2211.0,"1,77","23,77",France,NaN,Human-Computer Interaction; Language and Lingu...
216,0.540677,19_left,3157_right,NaN,"[Tien Ho-Phuoc, Anne Guérin-Dugué, Nathalie Gu...",00fd4f6f-c7a1-49a2-a029-d583d1d7a8b1,3,NaN,A COMPUTATIONAL SALIENCY MODEL INTEGRATING SAC...,international conference on bio-inspired syste...,...,0.0,56.0,0.0,31.0,54.0,"0,57","0,00",Portugal,NaN,Artificial Intelligence; Building and Construc...
519,0.357970,30_left,504_right,NaN,"[Olivia Mendoza, Patricia Melin, Oscar Castill...",016eba4d-f912-4927-874b-b0679bea78d2,50,"[159fdfd2-921e-4258-9e28-1b2c030e840e, 1a1e264...",Comparison of Fuzzy Edge Detectors Based on th...,soft computing,...,880.0,817.0,18950.0,1697.0,786.0,"2,04","21,53",Germany,Springer Verlag,Geometry and Topology (Q2); Software (Q2); The...
538,1.341802,31_left,5735_right,In system operations the term rollback is ofte...,"[Mark Burgess, Alva L. Couch]",01705f09-d395-4a0f-a89f-8d18382c81cc,8,"[363278e2-6c9d-45f5-90cd-0c4aaeb36631, 3a6f322...",On system rollback and totalized fields: An al...,The Journal of Logic and Algebraic Programming,...,0.0,4.0,0.0,1.0,4.0,"0,00","0,00",Netherlands,Elsevier BV,Computational Theory and Mathematics (Q4); Log...


In [49]:
score_null_count = dblp_jr_df['best_match_score'].isnull().sum()
score_null_count

529939

In [50]:
dblp_jr_df.columns

Index(['best_match_score', '__id_left', '__id_right', 'abstract', 'authors',
       'id', 'n_citation', 'references', 'title', 'venue', 'year', 'Rank',
       'Sourceid', 'Title', 'Type', 'Issn', 'SJR', 'SJR Best Quartile',
       'H index', 'Total Docs. (2017)', 'Total Docs. (3years)', 'Total Refs.',
       'Total Cites (3years)', 'Citable Docs. (3years)',
       'Cites / Doc. (2years)', 'Ref. / Doc.', 'Country', 'Publisher',
       'Categories'],
      dtype='object')

In [52]:
dblp_jr_df['Categories'].value_counts()

Software                                                                                                                                                                                                                                    58855
Computer Science (miscellaneous) (Q2); Theoretical Computer Science (Q3)                                                                                                                                                                    41029
Computer Networks and Communications; Hardware and Architecture; Software                                                                                                                                                                   34785
Computer Networks and Communications                                                                                                                                                                                                        31128
Electrical and Electronic Engine

In [54]:
oacc_df = pd.read_csv('open_access_cs.csv', sep=';')

In [ ]:
oacc_df['Issn'].value_counts()

In [57]:
dblp_jr_df['Issn']

0                          -
7                          -
10                  01628828
20                       NaN
21                       NaN
22                         -
24                       NaN
25                         -
50                         -
51                         -
62                       NaN
63                       NaN
64                         -
209                      NaN
210                 10450823
211                 15710661
212                        -
214                 19909772
215                      NaN
216                        -
218                 19994893
351       18757294, 09215034
396                 15206130
397                        -
404                      NaN
405                      NaN
406                      NaN
407                      NaN
408                        -
518                      NaN
                 ...        
969596                     -
969597                   NaN
969598              08912513
969613        